In [1]:
import requests
import pandas as pd

In [3]:
service_key = ' l2SER5UdFnVMTCfW/75Op1w1xj0ZVJyWjxZCA4s2qcMkJIjXonsQeorj8k35mmylOV0ywqc75RoXXtagwXuUnQ=='

##### 공공 포탈에서 부산광역시 행정 공공기관 데이터 불러오기

In [35]:
url = 'http://apis.data.go.kr/6260000/BusanTblPbaStusService/getTblPbaStusInfo'
params ={'serviceKey' : service_key, 'pageNo' : '1', 'numOfRows' : '360', 'resultType' : 'json'}

response = requests.get(url, params=params)
response.content

b'{"getTblPbaStusInfo":{"header":{"resultCode":"00","resultMsg":"NORMAL_CODE"},"body":{"items":{"item":[{"dept":"\xeb\xb0\x98\xec\x97\xac3\xeb\x8f\x99\xed\x96\x89\xec\xa0\x95\xeb\xb3\xb5\xec\xa7\x80\xec\x84\xbc\xed\x84\xb0","addrRoad":"\xeb\xb6\x80\xec\x82\xb0\xea\xb4\x91\xec\x97\xad\xec\x8b\x9c \xed\x95\xb4\xec\x9a\xb4\xeb\x8c\x80\xea\xb5\xac \xec\x9e\xac\xeb\xb0\x98\xeb\xa1\x9c256\xeb\xb2\x88\xea\xb8\xb8 29-70","tel":"051-749-6401","lat":"35.20099272","lng":"129.1335059","posCd":"48025","gugun":"\xeb\xb6\x80\xec\x82\xb0\xea\xb4\x91\xec\x97\xad\xec\x8b\x9c \xed\x95\xb4\xec\x9a\xb4\xeb\x8c\x80\xea\xb5\xac","dataDay":"2023-12-31"},{"dept":"\xed\x95\xb4\xec\x9a\xb4\xeb\x8c\x80\xea\xb5\xac \xeb\xac\xb8\xed\x99\x94\xeb\xb3\xb5\xed\x95\xa9\xec\x84\xbc\xed\x84\xb0","addrRoad":"\xeb\xb6\x80\xec\x82\xb0\xea\xb4\x91\xec\x97\xad\xec\x8b\x9c \xed\x95\xb4\xec\x9a\xb4\xeb\x8c\x80\xea\xb5\xac \xec\x84\xbc\xed\x85\x80\xec\xa4\x91\xec\x95\x99\xeb\xa1\x9c 170","tel":"051-749-6153","lat":"35.18218753","

In [37]:
## DataFrame으로 만든 후  컬럼 명 바꿔어서excel파일로 저장하기
data = response.json()
df_data = pd.DataFrame(data['getTblPbaStusInfo']['body']['items']['item'])

df_data.columns = ['공공기관명','도로명주소','전화번호','위도','경도','우편번호','구군명','등록일자']
df_data.to_excel('./data/부산광역시_행정공공기관.xlsx',index=False)

In [38]:
df_data.tail()

,공공기관명,도로명주소,전화번호,위도,경도,우편번호,구군명,등록일자
341,부산광역시 강서구청,부산광역시 강서구 낙동북로 477(대저1동),051-970-4000,35.212199,128.980589,46702,부산광역시 강서구,2024-01-01
342,강서산업단지관리사업소,부산광역시 강서구 녹산산단262로50번길 53(송정동),051-970-4832,35.084408,128.840245,46757,부산광역시 강서구,2024-01-01
343,가덕도동 행정복지센터,부산광역시 강서구 동선길 10(성북동),051-970-4637,35.05664,128.831651,46769,부산광역시 강서구,2024-01-01
344,명지2동 행정복지센터,부산광역시 강서구 명지오션시티11로 35(명지동),051-970-2984,35.084564,128.900068,46762,부산광역시 강서구,2024-01-01
345,명지1동 행정복지센터,부산광역시 강서구 영강길19번길 58(명지동),051-970-4924,35.108163,128.926325,46725,부산광역시 강서구,2024-01-01


In [39]:
## 구군명에서 '부산광역시' 삭제하기

for index,content in df_data.iterrows():
    df_data.loc[index,'구군명'] = content['구군명'].split(' ')[1]

In [40]:
## 필요 없는 칼럼 삭제하기

df_data.drop(['도로명주소','전화번호','우편번호','등록일자'],axis=1,inplace=True)

In [44]:
## 저장하기
df_data.to_excel('./data/행정기관_전처리.xlsx',index=False)

In [46]:
## 다시 읽어오기

df_data = pd.read_excel('./data/행정기관_전처리.xlsx')

In [50]:
## 구군별 행정기관 개수로 DataFrame만들기
gugun_count = pd.DataFrame(df_data['구군명'].value_counts())

In [55]:
gugun_count.reset_index(inplace=True)

In [56]:
gugun_count.head()

,구군명,count
0,금정구,54
1,사상구,42
2,연제구,39
3,사하구,39
4,해운대구,26


In [59]:
## 저장하기
gugun_count.to_excel('./data/구군별_행정기관갯수.xlsx',index=False)

##### 공공기관 지도 시각화

In [7]:
import json
import folium

In [ ]:
busan_loc = [35.1856505,129.1]
map = folium.Map(location=busan_loc,zoom_start=11)

for idx in range(0,len(df_data)):
    lat = df_data.loc[idx,'위도']
    lng = df_data.loc[idx,'경도']
    folium.CircleMarker(location=(lat,lng),popup=df_data.loc[idx,'공공기관명'],fill=True,fill_color='green',
                fill_opacity=2,color='yellow',radius=4,weight=1).add_to(seoulMap)